# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/weather_csv.csv")
weather_df

,Lat,Long,City,Country,Date,Max Temp,Cloudiness,Humidity,Wind Speed
0,72.7868,-56.1549,Upernavik,GL,1665505034,19.83,100,69,4.45
1,-46.1927,168.8643,Mataura,NZ,1665504745,43.48,99,65,0.78
2,8.5711,81.2335,Trincomalee,LK,1665505034,80.56,46,80,5.68
3,27.9769,-114.0611,Guerrero Negro,MX,1665505035,70.77,98,77,9.71
4,35.4333,133.8167,Kurayoshi,JP,1665505035,59.38,85,73,2.89
...,...,...,...,...,...,...,...,...,...
571,46.6166,10.7002,Laas,IT,1665505265,63.48,100,75,2.17
572,40.8106,111.6522,Hohhot,CN,1665505248,34.09,0,40,2.24
573,-28.2400,-48.6703,Imbituba,BR,1665505265,67.86,100,91,2.84
574,54.1502,-115.6855,Whitecourt,CA,1665505266,40.24,20,60,16.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_df[["Lat", "Long"]]
humidity = weather_df["Humidity"].astype(float)

max_intensity = weather_df["Humidity"].max()
fig = gmaps.figure(center=(25,0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=max_intensity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#drill drill drill
#temp
perf_temp = weather_df[weather_df["Max Temp"] > 70]
perf_temp = perf_temp[perf_temp["Max Temp"] < 80]
#wind
perf_wind = perf_temp[perf_temp["Wind Speed"] < 10]
#clouds
perf_places = perf_wind[perf_wind["Cloudiness"] == 0]
perf_places

,Lat,Long,City,Country,Date,Max Temp,Cloudiness,Humidity,Wind Speed
5,22.0752,-159.3190,Kapaa,US,1665505036,75.18,0,81,5.75
53,36.8163,50.8738,Tonekābon,IR,1665505055,71.55,0,65,4.29
70,-13.2000,49.0500,Ambilobe,MG,1665505061,72.46,0,67,2.28
110,31.6725,38.6637,Turaif,SA,1665505079,79.05,0,34,9.22
136,42.7762,-9.0603,Muros,ES,1665505089,76.84,0,66,9.71
149,35.2784,-93.1338,Russellville,US,1665505095,79.18,0,53,3.44
212,36.2946,-76.2511,Elizabeth City,US,1665505120,72.03,0,54,6.91
229,14.4000,46.4667,Marzuq,YE,1665505126,78.57,0,26,5.06
252,43.1548,-77.6156,Rochester,US,1665505068,70.72,0,58,5.75
285,-19.9842,57.6142,Cap Malheureux,MU,1665505148,71.47,0,68,5.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = perf_places.copy()
hotel_df = hotel_df.reset_index()
del hotel_df["index"]

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,Lat,Long,City,Country,Date,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
0,22.0752,-159.3190,Kapaa,US,1665505036,75.18,0,81,5.75,
1,36.8163,50.8738,Tonekābon,IR,1665505055,71.55,0,65,4.29,
2,-13.2000,49.0500,Ambilobe,MG,1665505061,72.46,0,67,2.28,
3,31.6725,38.6637,Turaif,SA,1665505079,79.05,0,34,9.22,
4,42.7762,-9.0603,Muros,ES,1665505089,76.84,0,66,9.71,
5,35.2784,-93.1338,Russellville,US,1665505095,79.18,0,53,3.44,
6,36.2946,-76.2511,Elizabeth City,US,1665505120,72.03,0,54,6.91,
7,14.4000,46.4667,Marzuq,YE,1665505126,78.57,0,26,5.06,
8,43.1548,-77.6156,Rochester,US,1665505068,70.72,0,58,5.75,
9,-19.9842,57.6142,Cap Malheureux,MU,1665505148,71.47,0,68,5.01,


In [13]:
#oh look another loop
lat = hotel_df["Lat"]
long = hotel_df["Long"]
range_x = 10

for x in range(range_x):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat[x]},{long[x]}", 
        "radius": 5000,
        "type": "lodging",
        "keyword": "Hotel",
        "key": g_key,
    }
    response = requests.get(base_url, params=params).json()
    
    try:
        results = response["results"][0]["name"]
        print(f"The closest hotel is {results}.")
        hotel_df.loc[hotel_df["City"] == hotel_df["City"][x], "Hotel Name"] = results        
    except (IndexError):
        print("No hotels within 5000 meters, skipping.")
    print("----")

The closest hotel is Kauai Shores Hotel.
----
The closest hotel is هتل.
----
The closest hotel is MOUNA HOTEL.
----
The closest hotel is Le Park Concord Turayf.
----
The closest hotel is Mar de Muros.
----
The closest hotel is Courtyard by Marriott Russellville.
----
The closest hotel is Fairfield Inn & Suites by Marriott Elizabeth City.
----
No hotels within 5000 meters, skipping.
----
The closest hotel is Holiday Inn Rochester NY - Downtown, an IHG Hotel.
----
The closest hotel is Lagoon Attitude Hotel.
----


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))